In [8]:
import zipfile
import os
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA

In [3]:

# Paths to the zip files
train_zip = '/kaggle/input/dogs-vs-cats/train.zip'
test_zip = '/kaggle/input/dogs-vs-cats/test1.zip'

# Extract directories
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

# Unzipping the training dataset
with zipfile.ZipFile(train_zip, 'r') as zip_ref:
    zip_ref.extractall(train_dir)

# Unzipping the test dataset
with zipfile.ZipFile(test_zip, 'r') as zip_ref:
    zip_ref.extractall(test_dir)

print("Data unzipped!")

Data unzipped!


In [4]:
# Paths to the directories
train_dir = '/kaggle/working/train/train'
test_dir = '/kaggle/working/test'

# Check the files in the train and test directories
train_images = os.listdir(train_dir)
test1_dir = os.path.join(test_dir, 'test1')

# Check contents of test1
test_images = os.listdir(test1_dir)

# Print the number of images and a few examples to verify
print(f"Number of training images: {len(train_images)}")
print(f"Number of test images: {len(test_images)}")

# Display a few examples
print("First 5 training images:", train_images[:5])
print("First 5 test images:", test_images[:5])


Number of training images: 25000
Number of test images: 12500
First 5 training images: ['cat.12225.jpg', 'cat.8047.jpg', 'cat.8037.jpg', 'dog.11216.jpg', 'dog.11261.jpg']
First 5 test images: ['10377.jpg', '6213.jpg', '5471.jpg', '5134.jpg', '3819.jpg']


In [5]:

# Function to load and preprocess images
def load_and_preprocess_images(image_paths, target_size=(128, 128)):
    images = []
    for image_path in image_paths:
        # Load image using OpenCV
        img = cv2.imread(image_path)
        
        # Check if the image was loaded successfully
        if img is None:
            print(f"Warning: Could not load image {image_path}")
            continue
        
        # Resize the image
        img_resized = cv2.resize(img, target_size)
        
        # Normalize pixel values to [0, 1]
        img_normalized = img_resized / 255.0
        
        # Flatten the image to a 1D array (for SVM)
        img_flattened = img_normalized.flatten()
        
        images.append(img_flattened)
    
    return np.array(images)

# Correct path to training images
train_dir = '/kaggle/working/train/train'
train_image_paths = [os.path.join(train_dir, img) for img in train_images]
X_train = load_and_preprocess_images(train_image_paths)

# Check the shape of the first preprocessed image
print(f"Shape of first preprocessed image: {X_train[0].shape}")
print(f"Total number of training images: {X_train.shape[0]}")

# Correct path to test images
test1_dir = '/kaggle/working/test/test1'
test_image_paths = [os.path.join(test1_dir, img) for img in test_images]
X_test = load_and_preprocess_images(test_image_paths)

print(f"Total number of test images: {X_test.shape[0]}")

Shape of first preprocessed image: (49152,)
Total number of training images: 25000
Total number of test images: 12500


In [6]:

# Step 1: Extract labels
def extract_labels(image_paths):
    labels = []
    for path in tqdm(image_paths, desc="Extracting labels"):
        filename = os.path.basename(path)
        if filename.startswith('cat'):
            labels.append(0)  # 0 for cats
        elif filename.startswith('dog'):
            labels.append(1)  # 1 for dogs
        else:
            print(f"Warning: Unknown label in filename {filename}")
    return np.array(labels)

# Extract labels for training images
y_train = extract_labels(train_image_paths)

# Step 2: Split into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train_split.shape[0]}")
print(f"Validation set size: {X_val.shape[0]}")

# Optional: Reduce dimensionality with IncrementalPCA in batches
print("Applying IncrementalPCA to reduce feature dimensions...")
batch_size = 16  # Use smaller batch size
ipca = IncrementalPCA(n_components=50)  # Use IncrementalPCA instead of PCA

# Fit IncrementalPCA incrementally in batches
for start in tqdm(range(0, len(X_train_split), batch_size), desc="Fitting IncrementalPCA"):
    end = min(start + batch_size, len(X_train_split))
    ipca.partial_fit(X_train_split[start:end])

# Transform training and validation data in batches
def transform_in_batches(data, ipca, batch_size):
    transformed_data = []
    for start in tqdm(range(0, len(data), batch_size), desc="Transforming data"):
        end = min(start + batch_size, len(data))
        transformed_data.append(ipca.transform(data[start:end]))
    return np.vstack(transformed_data)

X_train_pca = transform_in_batches(X_train_split, ipca, batch_size)
X_val_pca = transform_in_batches(X_val, ipca, batch_size)

# Step 3: Train the SVM classifier
print("Training the SVM classifier...")
start_time = time.time()

# SVM trains on the full PCA-transformed data (batch training not required for SVM)
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train_pca, y_train_split)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Step 4: Evaluate on the validation set
val_accuracy = svm_model.score(X_val_pca, y_val)
print(f"Validation Accuracy: {val_accuracy:.2f}")

Extracting labels: 100%|██████████| 25000/25000 [00:00<00:00, 1008168.60it/s]


Training set size: 20000
Validation set size: 5000
Applying PCA to reduce feature dimensions...


Fitting PCA:   0%|          | 0/1250 [00:00<?, ?it/s]


AttributeError: 'PCA' object has no attribute 'partial_fit'